In [2]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas_usaddress
# Load .env enviroment variables into the notebook
load_dotenv()
# Get the postgres connection information from os file. 

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import re
import time 

In [3]:

real_acct_df = pd.read_csv('../real_estate_data/real_acct_owner/real_acct.txt', sep='\t',lineterminator='\r',header=0,encoding='latin1',low_memory=False)

In [4]:
real_acct_df.head(2)

,acct,yr,mailto,mail_addr_1,mail_addr_2,mail_city,mail_state,mail_zip,mail_country,undeliverable,...,protested,certified_date,rev_dt,rev_by,new_own_dt,lgl_1,lgl_2,lgl_3,lgl_4,jurs
0,\n0010010000013,2021.0,CITY OF HOUSTON,PO BOX 1562,,HOUSTON,TX,77251-1562,,N,...,N,08/06/2021,01/01/2016,01391,01/02/1988,ALL BLK 1,SSBB,NaN,NaN,001 040 041 042 043 044 048 061 265 268 576
1,\n0010020000001,2021.0,CURRENT OWNER,1717 SAINT JAMES PLACE STE 112,,HOUSTON,TX,77056-3412,,N,...,Y,NaN,05/28/2020,00203,10/31/2012,TR 15 BLK 2,SSBB,NaN,NaN,001 040 041 042 043 044 048 061 265 268 576


In [5]:
columns = real_acct_df.columns
columns

Index(['acct', 'yr', 'mailto', 'mail_addr_1', 'mail_addr_2', 'mail_city',
       'mail_state', 'mail_zip', 'mail_country', 'undeliverable', 'str_pfx',
       'str_num', 'str_num_sfx', 'str', 'str_sfx', 'str_sfx_dir', 'str_unit',
       'site_addr_1', 'site_addr_2', 'site_addr_3', 'state_class',
       'school_dist', 'map_facet', 'key_map', 'Neighborhood_Code',
       'Neighborhood_Grp', 'Market_Area_1', 'Market_Area_1_Dscr',
       'Market_Area_2', 'Market_Area_2_Dscr', 'econ_area', 'econ_bld_class',
       'center_code', 'yr_impr', 'yr_annexed', 'splt_dt', 'dsc_cd', 'nxt_bld',
       'bld_ar', 'land_ar', 'acreage', 'Cap_acct', 'shared_cad', 'land_val',
       'bld_val', 'x_features_val', 'ag_val', 'assessed_val', 'tot_appr_val',
       'tot_mkt_val', 'prior_land_val', 'prior_bld_val',
       'prior_x_features_val', 'prior_ag_val', 'prior_tot_appr_val',
       'prior_tot_mkt_val', 'new_construction_val', 'tot_rcn_val',
       'value_status', 'noticed', 'notice_dt', 'protested', 'certif

In [6]:
# new_cols = ['acct', 'str_pfx', 'str_num',
#        'str_num_sfx', 'str', 'str_sfx', 'str_sfx_dir', 'str_unit',
#        'site_addr_1', 'site_addr_2', 'site_addr_3','bld_val', 'x_features_val', 'ag_val', 
#        'assessed_val', 'tot_appr_val',
#        'tot_mkt_val', 'prior_land_val', 'prior_bld_val',
#        'prior_x_features_val', 'prior_ag_val', 'prior_tot_appr_val',
#        'prior_tot_mkt_val', 'new_construction_val', 'tot_rcn_val',
#        'value_status', 'certified_date']

In [7]:
new_cols = ['acct','str_num','str_num_sfx','str','str_sfx','str_sfx_dir','str_unit','site_addr_1', 'site_addr_2',
'site_addr_3','bld_ar','land_ar','tot_appr_val','tot_mkt_val']

In [8]:
# remove the columns that we don't need
real_acct_df = real_acct_df[new_cols]
real_acct_df.head()

,acct,str_num,str_num_sfx,str,str_sfx,str_sfx_dir,str_unit,site_addr_1,site_addr_2,site_addr_3,bld_ar,land_ar,tot_appr_val,tot_mkt_val
0,\n0010010000013,0.0,,COMMERCE,ST,,,0 COMMERCE ST,HOUSTON,77002,0.0,44431.0,0.0,0.0
1,\n0010020000001,907.0,,COMMERCE,ST,,,907 COMMERCE ST,HOUSTON,77002,0.0,5001.0,309120.0,309120.0
2,\n0010020000003,0.0,,COMMERCE,ST,,,0 COMMERCE ST,HOUSTON,77002,0.0,18121.0,889398.0,889398.0
3,\n0010020000004,0.0,,COMMERCE,ST,,,0 COMMERCE ST,HOUSTON,77002,0.0,9061.0,444699.0,444699.0
4,\n0010020000013,921.0,,COMMERCE,ST,,,921 COMMERCE ST,HOUSTON,77002,0.0,3001.0,0.0,0.0


In [9]:
# remove the \n from the account column
def clean(x):
    x = x.replace('\n','')
    return x

In [10]:
real_acct_df['acct'] = real_acct_df['acct'].apply(clean)

In [11]:

real_acct_df.dtypes

acct             object
str_num         float64
str_num_sfx      object
str              object
str_sfx          object
str_sfx_dir      object
str_unit         object
site_addr_1      object
site_addr_2      object
site_addr_3      object
bld_ar          float64
land_ar         float64
tot_appr_val    float64
tot_mkt_val     float64
dtype: object

In [12]:
real_acct_df.head(2)

,acct,str_num,str_num_sfx,str,str_sfx,str_sfx_dir,str_unit,site_addr_1,site_addr_2,site_addr_3,bld_ar,land_ar,tot_appr_val,tot_mkt_val
0,0010010000013,0.0,,COMMERCE,ST,,,0 COMMERCE ST,HOUSTON,77002,0.0,44431.0,0.0,0.0
1,0010020000001,907.0,,COMMERCE,ST,,,907 COMMERCE ST,HOUSTON,77002,0.0,5001.0,309120.0,309120.0


In [13]:
# load 5 million rows to local database
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:5432/Houston_real_estate')


In [14]:
# load df to database
real_acct_df.to_sql('real_acct',engine,index=False,if_exists='replace')

## Start Stan's web scraping from HAR

In [15]:
def fix_num(num):
    return float(re.sub('[^0-9\.]', "", num))

def fix_num_int(num):
    return int(re.sub('[^0-9\.]', "", num))

In [16]:
#read Haris county zip codes
zip_code_data=pd.read_csv("harris_county_zip_codes.csv")
zip_code_list=list(zip_code_data['zip_code'].map(str))

In [17]:
#set up chrome browser
# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)

In [18]:
# #define empty dataframe to appand data to
# har_df = pd.DataFrame(columns = ['mls', 'address', 'bld_sft','lot_sft','price', 'zip'])
# #search patterns for building and land square footage
# bldgsft_pattern = re.compile(r'Building Sqft')
# lotsft_pattern = re.compile(r'Lot Sqft')

# #define a subroutine to extract data from house structure and append it to a dataframe
# def extract_from_house(house):
#     global har_df
#     asking_price=fix_num(house.find('div', class_="price").text)
#     try:
#         bldg_sft=fix_num(house.find('div', class_="mp_features").find(text=bldgsft_pattern).__dict__["previous_sibling"].text)
#     except AttributeError:
#         bldg_sft=0
#     try:
#         lot_sft=fix_num(house.find('div', class_="mp_features").find(text=lotsft_pattern).__dict__["previous_sibling"].text)
#     except AttributeError:
#         lot_sft=0
#     addr=house.find('a', class_="address").text
#     mls_no=fix_num_int(house.find('div', class_="mpi_mls").text)
#     property_zip=current_zip
#     har_df = har_df.append({'mls' : mls_no, 'address' : addr, 'bld_sft' : bldg_sft, 'lot_sft': lot_sft, 'price': asking_price, 'zip': current_zip }, ignore_index = True)

# # cycle through all Harris county zip codes
# for current_zip in zip_code_list:
#     time.sleep(10)
#     print(current_zip)
#     #define starting url for current zip code
#     url=f'https://www.har.com/zipcode_{current_zip}/realestate/for_sale'
#     browser.visit(url)
#     html = browser.html
#     soup = bs(html, 'html.parser')
#     #Scrape the houses for the current search page
#     houses=soup.find_all('div', class_="prop_item")
        
#     for house in houses:
#         extract_from_house(house)

#     #figure out how many pages we will have to click through
#     navigation=soup.find_all('div', class_="pagination")
#     number_of_pages=len(navigation[0].ul.find_all('li'))

#     #click through pages 2 through the last page 
#     for i in range(2, number_of_pages+1):
#         nextlink=browser.links.find_by_text(str(i))
#         time.sleep(5)
#         #figure out ambiguity to what is the next search page link
#         if len(nextlink)==1:
#             nextlink.click()
#         else:
#             nextlink[1].click()
        
#         #extract houses from the current page and append to a data frame
#         html = browser.html
#         soup = bs(html, 'html.parser')
#         houses=soup.find_all('div', class_="prop_item")
#         for house in houses:
#             extract_from_house(house)
            
# #save scraped results for later use            
# har_df.to_csv('har_data.csv',index=False)

In [19]:
# current_zip='77401'
# bldgsft_pattern = re.compile(r'Building Sqft')
# lotsft_pattern = re.compile(r'Lot Sqft')
# for house in houses:
#     asking_price=fix_num(house.find('div', class_="price").text)
#     bldg_sft=fix_num(house.find('div', class_="mp_features").find(text=bldgsft_pattern).__dict__["previous_sibling"].text)
#     lot_sft=fix_num(house.find('div', class_="mp_features").find(text=lotsft_pattern).__dict__["previous_sibling"].text)
#     addr=house.find('a', class_="address").text
#     mls_no=fix_num_int(house.find('div', class_="mpi_mls").text)
#     property_zip=current_zip
#     har_df = har_df.append({'mls' : mls_no, 'address' : addr, 'bld_sft' : bldg_sft, 'lot_sft': lot_sft, 'price': asking_price, 'zip': current_zip }, ignore_index = True)
    

In [20]:
# Load cities to a dataframe
har = pd.read_csv('har_data.csv')
har_df = pd.DataFrame(har)
har_df.head()


,mls,address,bld_sft,lot_sft,price,zip
0,17777272,"1211 Caroline Street PH2002, Houston, TX 77002",2433.0,0.0,1745000.0,77002
1,54714244,"1211 Caroline Street #1903, Houston, TX 77002",2579.0,0.0,1490000.0,77002
2,8077568,"915 Franklin Street 9F, Houston, TX 77002",2380.0,0.0,1170000.0,77002
3,43392628,"2000 Bagby Street #15400, Houston, TX 77002",2359.0,0.0,1100000.0,77002
4,64069960,"1211 Caroline st #1105, Houston, TX 77002",1235.0,0.0,650000.0,77002


In [21]:
# Parse address into multiple columns to enable query with hcad data
address_df = pandas_usaddress.tag(har_df, ['address'])
address_df.head(2)

,mls,address,bld_sft,lot_sft,price,zip,AddressNumber,AddressNumberPrefix,AddressNumberSuffix,BuildingName,...,StreetNamePostDirectional,StreetNamePostModifier,StreetNamePostType,SubaddressIdentifier,SubaddressType,USPSBoxGroupID,USPSBoxGroupType,USPSBoxID,USPSBoxType,ZipCode
0,17777272,"1211 Caroline Street PH2002, Houston, TX 77002",2433.0,0.0,1745000.0,77002,1211,NaN,NaN,NaN,...,NaN,NaN,street,NaN,NaN,NaN,NaN,NaN,NaN,77002
1,54714244,"1211 Caroline Street #1903, Houston, TX 77002",2579.0,0.0,1490000.0,77002,1211,NaN,NaN,NaN,...,NaN,NaN,street,NaN,NaN,NaN,NaN,NaN,NaN,77002


In [22]:
# load to postgre
address_df.to_sql('har_address',engine,index=False,if_exists='replace')


In [23]:
# load to postgre
har_df.to_sql('har',engine,index=False,if_exists='replace')

In [24]:
# change column data type to text
engine.execute('''
alter table real_acct 
alter column str_num type varchar;
''')

In [26]:
# query postgres and store in dataframe
har_vs_hcad_df = pd.read_sql('''
select ha."mls", r.str_num,r.str,r.site_addr_1,r.site_addr_3,ha.price as "har price", r.tot_appr_val as "hcad appraised value",
r.tot_mkt_val as "hcad total mkt value"
from har_address ha, real_acct r
where ha."ZipCode" = r.site_addr_3
and upper(ha."StreetName") = upper(r.str)
and ha."AddressNumber" = r.str_num''',engine)

In [27]:
har_vs_hcad_df

,mls,str_num,str,site_addr_1,site_addr_3,har price,hcad appraised value,hcad total mkt value
0,11977274,0,BELL,0 BELL ST,77003,130000.0,82072.0,82072.0
1,11977274,0,BELL,0 BELL ST,77003,130000.0,83600.0,83600.0
2,11977274,0,BELL,0 BELL ST,77003,130000.0,84700.0,84700.0
3,11977274,0,BELL,0 BELL ST,77003,130000.0,81620.0,81620.0
4,11977274,0,BELL,0 BELL ST,77003,130000.0,125000.0,125000.0
...,...,...,...,...,...,...,...,...
147665,19669273,9919,ORCHID SPRING,9919 ORCHID SPRING LN,77044,220000.0,182783.0,193411.0
147666,95592221,9922,PENNYMILL,9922 PENNYMILL DR,77396,399500.0,315593.0,315593.0
147667,33182545,9927,SAGEROCK,9927 SAGEROCK DR,77089,225000.0,216911.0,216911.0
147668,81113544,9934,RUSTIC GATE,9934 RUSTIC GATE RD,77571,254900.0,198290.0,198290.0


In [ ]:
# end session and close the browser
browser.quit()